In [50]:
import MetaTrader5 as mt5
import pandas as pd
import time
from datetime import datetime
import ta

In [2]:

def connect_to_mt5():
    if not mt5.initialize():
        print("Failed to initialize the Metatrader 5 library.")
        return False
    print("Connected to MetaTrader 5")
    return True

In [14]:

def get_data(symbol, timeframe, start_date, end_time):
    rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_time)
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    return df

In [4]:

def get_sma(data, period):
    calc_sma = data['close'].tail(period)
    sma = calc_sma.mean()
    return sma


In [5]:
def get_prev_sma(data, period):
    prev_sma = data['close'].rolling(period).mean()
    return prev_sma


In [6]:
def market_order(symbol, volume, order_type, deviation=20, magic=261200):
    order_type_dict = {
        'buy'  : mt5.ORDER_TYPE_BUY,
        'sell'  : mt5.ORDER_TYPE_SELL
    }
    price_dict = {
        'buy' : mt5.symbol_info_tick(symbol).ask,
        'sell' : mt5.symbol_info_tick(symbol).bid
    }
    
    if order_type == 'buy':
        sl_price = price_dict['buy'] - (200 * mt5.symbol_info(symbol).point)
        tp_price = price_dict['buy'] + (500 * mt5.symbol_info(symbol).point)
    elif order_type == 'sell':
        sl_price = price_dict['sell'] + (200 * mt5.symbol_info(symbol).point)
        tp_price = price_dict['sell'] - (500 * mt5.symbol_info(symbol).point)
    else:
        raise ValueError("Invalid order type.")
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,  # FLOAT
        "type": order_type_dict[order_type],
        "price": price_dict[order_type],
        "sl": sl_price,  # FLOAT
        "tp": tp_price,  # FLOAT
        "deviation": deviation,  # INTEGER
        "magic": magic,  # INTEGER
        "comment": "my_first_strat",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    
    order_result = mt5.order_send(request)
    return order_result

In [10]:
def main():
    symbol = 'XAUUSD'
    timeframe = mt5.TIMEFRAME_M15
    volume = 0.1
    
    if not connect_to_mt5():
        return
    
    while True:
        account_info = mt5.account_info()
        print(datetime.now(),
              '| Login: ', account_info.login,
              '| Balance: ', account_info.balance,
              '| Equity: ' , account_info.equity,
              '| Profit: ', account_info.profit)
        
        current_time = datetime.now()
        start_time = datetime(2024, 1, 1)
        end_time = current_time

        data = get_data(symbol, timeframe, start_time, end_time)
        if len(data) < 120:  
            print("Not enough data.")
            time.sleep(60)
            continue
        
        fast_ma = get_sma(data,  20)
        slow_ma = get_sma(data, 120)
        


        # Check if there are open positions
        positions_total = mt5.positions_total()
        
        # Keep track of current position
        in_position = positions_total > 0
        
        prev_fast_ma = get_prev_sma(data, 20).iloc[-2]
        prev_slow_ma = get_prev_sma(data, 120).iloc[-2]


        # Check conditions and place trades
        if not in_position: 
            if (fast_ma > slow_ma) and (prev_fast_ma <= prev_slow_ma):
                market_order(symbol, volume, 'buy')
                print('Buy signal detected')
                in_position = True

            elif (fast_ma < slow_ma) and (prev_fast_ma >= prev_slow_ma):
                market_order(symbol, volume, 'sell')
                print('Sell signal detected')
                in_position = True
            
        
        time.sleep(900)  # Check every 15 minutes

In [51]:
ta.trend.sma_indicator(data['close'], window=20)

time
2024-01-02 01:00:00          NaN
2024-01-02 01:15:00          NaN
2024-01-02 01:30:00          NaN
2024-01-02 01:45:00          NaN
2024-01-02 02:00:00          NaN
                         ...    
2024-03-27 13:15:00    2181.3170
2024-03-27 13:30:00    2182.0605
2024-03-27 13:45:00    2182.8985
2024-03-27 14:00:00    2183.6970
2024-03-27 14:15:00    2184.5785
Name: sma_20, Length: 5644, dtype: float64

In [63]:
time = datetime.utcfromtimestamp(mt5.symbol_info('XAUUSD').time)
time

datetime.datetime(2024, 3, 27, 17, 45, 27)

In [64]:
symbol = 'XAUUSD'
timeframe = mt5.TIMEFRAME_M15
current_time =  datetime.utcfromtimestamp(mt5.symbol_info('XAUUSD').time)
start_time = datetime(2024, 1, 1)
end_time = current_time
data = get_data(symbol, timeframe, start_time, end_time)

fast_ma = get_sma(data,  20)
slow_ma = get_sma(data, 120)

prev_fast_ma = get_prev_sma(data, 20).iloc[-2]
prev_slow_ma = get_prev_sma(data, 120).iloc[-2]

output1 = (fast_ma > slow_ma) and (prev_fast_ma <= prev_slow_ma)
output2 = (fast_ma < slow_ma) and (prev_fast_ma >= prev_slow_ma)

print(f'Fastma :{fast_ma}')
print(f'Slowma :{slow_ma}')
print(f'Prev Fastma :{prev_fast_ma}')
print(f'Prev Slowma :{prev_slow_ma}')
print(f'(fast_ma > slow_ma) and (prev_fast_ma <= prev_slow_ma) :{output1}')
print(f'(fast_ma < slow_ma) and (prev_fast_ma >= prev_slow_ma) :{output2}')

Fastma :2190.6150000000002
Slowma :2182.527416666667
Prev Fastma :2190.4530000000004
Prev Slowma :2182.365
(fast_ma > slow_ma) and (prev_fast_ma <= prev_slow_ma) :False
(fast_ma < slow_ma) and (prev_fast_ma >= prev_slow_ma) :False


In [ ]:
if __name__ == "__main__":
    main()